# Craete Sample Table

## 1. Create Sample Data

| gendor | date | sentence |
| --- | --- | --- |
| M | 2023-04-01 | quick quick the quick mysql |

In [5]:
import pandas as pd
import itertools
import random
import time
import nltk


# https://stackoverflow.com/questions/553303/generate-a-random-date-between-two-other-dates
def str_time_prop(start, end, time_format, prop):
    """Get a time at a proportion of a range of two formatted times.

    start and end should be strings specifying times formatted in the
    given format (strftime-style), giving an interval [start, end].
    prop specifies how a proportion of the interval to be taken after
    start.  The returned time will be in the specified format.
    """

    stime = time.mktime(time.strptime(start, time_format))
    etime = time.mktime(time.strptime(end, time_format))

    ptime = stime + prop * (etime - stime)

    return time.strftime(time_format, time.localtime(ptime))


def random_date(start, end, prop):
    return str_time_prop(start, end, "%Y-%m-%d", prop)


print(random_date("2020-01-01", "2023-12-31", random.random()))
# print(generate_sentence(cfd, initstr))

2022-10-18


In [12]:
def get_random_date():
    return random_date("2022-01-01", "2023-12-31", random.random())

def generate_sentence(length=5):
  """Generates a random sentence in English."""
  words = ["the", "quick", "brown", "fox", "jumps", "over", "the", "lazy", "dog", "man", "women", "car", "banana", "mysql", "database", "is", "a", "that"]
  return " ".join(random.choice(words) for _ in range(length))

print(generate_sentence())

mysql a jumps women a


In [13]:
SAMPLE_ROWS = 1000000

In [14]:
sample_data = []
for i in range(SAMPLE_ROWS):
    if i % 100000 == 0:
        print(i)
    sample_data.append(["M", get_random_date(), generate_sentence(10)])
    sample_data.append(["F", get_random_date(), generate_sentence(10)])

0
100000
200000
300000
400000
500000
600000
700000
800000
900000


In [28]:
col_list = ["gender", "date", "sentence"]

In [29]:
df = pd.DataFrame(sample_data, columns=col_list)

In [30]:
df.head()

,gender,date,sentence
0,M,2022-02-26,fox is car that dog man lazy the lazy women
1,F,2022-11-29,the a quick quick the women database the the that
2,M,2023-12-18,lazy database banana a over dog man over lazy car
3,F,2022-10-23,fox a that lazy car car women mysql car fox
4,M,2022-08-23,the man jumps the jumps quick fox the the the


## 2. Create Table

### 2.1. Create "gender_dates" Table

In [31]:
import pymysql
import sqlalchemy
from sqlalchemy import create_engine

pymysql.install_as_MySQLdb()
# date_after_month = datetime.today()+ relativedelta(months=1)
# ref: https://stackoverflow.com/questions/4130922/how-to-increment-datetime-by-custom-months-in-python-without-using-library

In [32]:
USER = "root"
PASSWORD = "1111"
ENDPOINT = "localhost"

In [33]:
conn = pymysql.connect(
    host=ENDPOINT,
    user=USER,
    password=PASSWORD,
    port=3306,
    cursorclass=pymysql.cursors.DictCursor,
)

cur = conn.cursor()

In [34]:
cur.execute("CREATE DATABASE IF NOT EXISTS real_mysql;")

1

In [35]:
engine = create_engine(f"mysql+mysqldb://{USER}:{PASSWORD}@{ENDPOINT}:3306")
# http://i5on9i.blogspot.com/2020/05/no-module-named-mysqldb.html

df.to_sql(
    "gender_dates",
    engine,
    schema="real_mysql",
    index=False,
    if_exists="replace",
    chunksize=10000,
)

In [37]:
cur.execute("alter table real_mysql.gender_dates modify `gender` varchar(10);")
cur.execute("alter table real_mysql.gender_dates modify `date` date;")
cur.execute("alter table real_mysql.gender_dates modify `sentence` varchar(255);")

2000000

### 2.2. Create "big_table" Table (gendor_dates x 10)

In [44]:
create_big_table = """
create table real_mysql.big_table
select * from real_mysql.gender_dates;
"""

In [45]:
insert_big_table = """
insert into real_mysql.big_table
select * from real_mysql.gendor_dates;
"""

In [46]:
cur.execute(create_big_table)

2000000

In [48]:
for _ in range(9):
    cur.execute(insert_big_table)